In [1]:
import numpy as np
import xarray as xr
import glob
import os
import hydromt
from dask.diagnostics import ProgressBar 

In [2]:
# setup logging
from hydromt.log import setuplog
logger = setuplog("read raster data", log_level=10)

2022-05-16 10:48:43,851 - read raster data - log - INFO - HydroMT version: 0.4.5


In [3]:
# Open data using the deltares data catalog
data_catalog = hydromt.DataCatalog(deltares_data=True, logger=logger)
data_catalog

2022-05-16 10:48:51,305 - read raster data - data_adapter - INFO - Adding deltares_data v0.0.6 sources from https://github.com/DirkEilander/hydromt-artifacts/releases/download/v0.0.6/deltares_data.yml


,path,data_type,driver,category,paper_doi,paper_ref,source_license,source_url,source_version,source_author,unit,history,notes,source_info,author,timestamp
chelsa,p:\wflow_global\hydromt\meteo\chelsa_clim_v1.2...,RasterDataset,raster,meteo,10.1038/sdata.2017.122,Karger et al. (2017),CC BY 4.0,http://chelsa-climate.org/downloads/,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chelsa_v1.2,p:\wflow_global\hydromt\meteo\chelsa_clim_v1.2...,RasterDataset,raster,meteo,10.1038/sdata.2017.122,Karger et al. (2017),CC BY 4.0,http://chelsa-climate.org/downloads/,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chirps,p:\wflow_global\hydromt\meteo\chirps_africa_da...,RasterDataset,netcdf,meteo,10.1038/sdata.2015.66,Funk et al (2015),CC,https://www.chc.ucsb.edu/data/chirps,v2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chirps_africa_daily_v2.0,p:\wflow_global\hydromt\meteo\chirps_africa_da...,RasterDataset,netcdf,meteo,10.1038/sdata.2015.66,Funk et al (2015),CC,https://www.chc.ucsb.edu/data/chirps,v2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chirps_global,p:\wflow_global\hydromt\meteo\chirps_global_da...,RasterDataset,netcdf,meteo,10.3133/ds832,Funk et al (2014),CC,https://www.chc.ucsb.edu/data/chirps,v2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
worldclim_v2.0,p:\wflow_global\hydromt\meteo\worldclim_v2.0\w...,RasterDataset,netcdf,meteo,10.1002/joc.5086,Fick and Hijmans (2017),NaN,https://www.worldclim.org/data/worldclim21.html,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
worldpop_2020_constrained,p:\wflow_global\hydromt\socio_economic\worldpo...,RasterDataset,raster,socio-economic,10.1371/journal.pone.0107042,Stevens et al. (2015),CC BY 4.0,https://www.worldpop.org/doi/10.5258/SOTON/WP0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
worldpop_2020_UNadj_constrained,p:\wflow_global\hydromt\socio_economic\worldpo...,RasterDataset,raster,socio-economic,10.1371/journal.pone.0107042,Stevens et al. (2015),CC BY 4.0,https://www.worldpop.org/doi/10.5258/SOTON/WP0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wsf_bld_2015,p:\wflow_global\hydromt\infrastructure\wsf\WSF...,RasterDataset,raster,socio-economic,10.1038/s41597-020-00580-5,Marconcini at al (2020),CC0 1.0,https://un-spider.org/links-and-resources/data...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# check out era5_daily
data_catalog["era5"]

{'category': 'meteo',
 'data_type': 'RasterDataset',
 'driver': 'netcdf',
 'history': 'Extracted from Copernicus Climate Data Store; resampled by '
            'Deltares to daily frequency',
 'paper_doi': '10.1002/qj.3803',
 'paper_ref': 'Hersbach et al. (2019)',
 'path': 'p:\\wflow_global\\hydromt\\meteo\\era5_daily\\nc_merged\\era5_*_daily.nc',
 'source_license': 'https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products',
 'source_url': 'https://doi.org/10.24381/cds.bd0915c6',
 'source_version': 'ERA5 daily data on pressure levels'}

In [5]:
# select start and end time and retrieve dataset
ts = "1981-01-01T00:00:00"
te = "2010-12-31T00:00:00" 
bbox = [97.76,13.1,101.96,20.5] # ymin xmin ymax xmax 
ds = data_catalog.get_rasterdataset("era5_daily", bbox=bbox, buffer=1, time_tuple=(ts, te))  # bbox=[0., 0., 2., 2.]
ds

2022-05-16 10:49:56,878 - read raster data - data_adapter - INFO - DataCatalog: Getting era5_daily RasterDataset netcdf data from p:\wflow_global\hydromt\meteo\era5_daily\nc_merged\era5_*_daily.nc
2022-05-16 10:50:01,802 - read raster data - data_adapter - DEBUG - RasterDataset: Slicing time dim ('1981-01-01T00:00:00', '2010-12-31T00:00:00')
2022-05-16 10:50:02,181 - read raster data - data_adapter - DEBUG - RasterDataset: Clip with bbox - [97.760, 13.100, 101.960, 20.500]
2022-05-16 10:50:02,195 - read raster data - data_adapter - DEBUG - RasterDataset: Convert units for 6 variables.


<xarray.Dataset>
Dimensions:      (time: 10957, longitude: 18, latitude: 32)
Coordinates:
  * time         (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2010-12-31
  * longitude    (longitude) float32 97.75 98.0 98.25 98.5 ... 101.5 101.8 102.0
  * latitude     (latitude) float32 20.75 20.5 20.25 20.0 ... 13.5 13.25 13.0
    spatial_ref  int32 1
Data variables:
    press_msl    (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    kin          (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    temp_min     (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    temp_max     (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    temp         (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    kout         (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
    precip       (time, latitude, longitude) float32 dask.array<chunksize=(1, 32, 18), meta=np.ndarray>
Attributes:
    category:        meteo
    history:         Extracted from Copernicus Climate Data Store; resampled ...
    paper_doi:       10.1002/qj.3803
    paper_ref:       Hersbach et al. (2019)
    source_license:  https://cds.climate.copernicus.eu/cdsapp/#!/terms/licenc...
    source_url:      https://doi.org/10.24381/cds.bd0915c6
    source_version:  ERA5 daily data on pressure levels

In [6]:
dvars = ds.raster.vars
encoding={k: {"zlib": True} for k in dvars}

print("Saving to netcdf")
delayed_obj = ds.to_netcdf("era5_bangkok_1981-2011.nc", encoding=encoding, mode="w", compute=False)
with ProgressBar():
    delayed_obj.compute() 

Saving to netcdf
[########################################] | 100% Completed | 16min 57.6s
